# DeepFool- Adversarial Attacks
### Paper link : https://arxiv.org/abs/1511.04599

## Objectives & Context

In the previous sections, we explored **FGSM, BIM, PGD, and Auto-PGD**. If you haven’t gone through these notebooks yet, we highly recommend reviewing them first, as **DeepFool is a fundamentally different approach**.

So far, we’ve seen:

- **FGSM** applies a single-step attack (fast but easy to defend against).
- **BIM** refines the attack by applying multiple smaller steps instead of one big step (stronger than FGSM but lacks control).
- **PGD** corrects BIM by projecting perturbations into a norm-constrained space to ensure they remain valid (more stable and harder to defend against).
- **Auto-PGD** dynamically adjusts `eps_step` during the attack process to make the attack more efficient.

### **Why do we need DeepFool? What problem does it solve?**

All previous **Gradient-Based Attacks** (FGSM, BIM, PGD) follow the same logic:
👉 They **move in the gradient direction** without thinking about the **best** way to cross the decision boundary.

**The Problem with PGD & Auto-PGD:**
- They **do not look at the geometry of the decision boundary**.
- They **blindly apply perturbations** in a fixed or adaptive manner but may use more perturbations than needed.
- They **waste steps and energy** by not optimizing the attack path.

### **DeepFool brings a new perspective!**
DeepFool is the first attack to focus on **finding the *minimal* perturbation** required to cross the decision boundary.

It answers the question:  **What is the smallest possible change I can apply to fool the model?**


DeepFool solves this problem by:
- **Finding the closest decision boundary** instead of blindly following the gradient.
- **Applying only the necessary perturbation** to cross this boundary.
- **Stopping immediately** once the adversarial image is misclassified.


## How Does DeepFool Stop Automatically?

Unlike PGD and Auto-PGD, **DeepFool doesn't require a fixed number of iterations**.

Instead, DeepFool:
1. **Checks the classification at each step (that's why it's a White-Box Attack ! It used the model to classify at each iteration)**.
2. **If the class has changed, it stops immediately**.
3. **If not, it keeps moving in the optimal direction**.

**This makes DeepFool one of the most efficient gradient-based attacks** because it **only applies the minimal necessary perturbation**.

*NOTE*: Imagine a scenario where DeepFool doesn't success to create adversarial image. It will run indefinitely. To avoid this scenario, we will put the parameter `max_iter` which will specify the maximum number of iteration. After this number, it will simply stop the process. Even another class is not reached.

------

### Weaknesses

However, because DeepFool **don't search to maximise the confidence score** of the error classification, it just takes de first bad classification get, even if :
* The target class has a **weak confidence** score (i,e. 30%)
* The original class is **still close** in the confidence score (i,e. 25%)

**Because, after the moment that the classification is wrong, DeepFool consider that his job is done !**

This weakness will be solve by another attack that we'll see later ;)

--> DeepFool is efficient to create **minimal perturbations**, but his adversarial examples can be **unstable and easy to correct** !

---

## Mathmatics Formula & Step Adaptation

Unlike PGD, which moves with a **fixed step size** or an **adaptive step size**, DeepFool continuously estimates **the minimal distance to the decision boundary**.

At each step **t**, the adversarial image is updated using the formula:

**$$x_{t+1} = x_t + \frac{- f(x_t)}{\|\nabla f(x_t) \|^2} \nabla f(x_t)$$**

Where:
- **$ x_t $** is the current adversarial image at iteration **t**.
- **$ f(x_t) $** is the model's classification score for the current class.
- **$ \nabla f(x_t) $** is the gradient of the model (the direction where classification changes the most).
- **$ \frac{- f(x_t)}{\|\nabla f(x_t) \|^2} $** Normalization factor to prevent overly large perturbations.

---

**Why do we divide by $\|\nabla f(x_t) \|^2$ ?**
Because it **normalizes** the gradient direction, making sure the step size is proportional to the confidence of the model, instead of just blindly following the steepest descent.

**Why do we multiply by $- f(x_t)$ ?**
This term **measures the distance to the decision boundary, so the perturbation is **scaled accordingly**. The closer we are to the boundary, the smaller the step.

---

### Example with a Single Neuron

Let’s take **a single neuron** in a binary classification model that distinguishes between **cats** and **dogs**.

#### **Initial Conditions:**
- **Input $x_t = 0.6$** → A feature value representing an image characteristic (e.g., pixel intensity, normalized between 0 and 1).
- **Score $f(x_t) = 0.2$** → The model still classifies it as a **cat**, but it is **close to the decision boundary**.
- **Gradient $\nabla f(x_t) = 0.5$** → The direction in which the score changes the most when modifying $x_t$.

#### **Applying the DeepFool Formula:**

$$x_{t+1} = x_t + \left(-\frac{f(x_t)}{||\nabla f(x_t)||^2}\right) \cdot \nabla f(x_t)$$


1. **Computing the correction factor** (how far we are from the decision boundary):  
$$
   -\frac{f(x_t)}{||\nabla f(x_t)||^2} = -\frac{0.2}{(0.5)^2} = -\frac{0.2}{0.25} = -0.8
$$

2. **Updating the input value \( x_t \)**:  
$$
   x_{t+1} = 0.6 + (-0.8 \times 0.5)
$$
$$
   x_{t+1} = 0.6 - 0.4
$$
$$
   \mathbf{x_{t+1} = 0.2}
$$

**Result:**  
The attack **efficiently moves** towards the decision boundary by reducing $x$, and the model might now misclassify the image as a **dog** instead of a **cat**. If not, we repeat another iteration by following the same process. The step size will be surely lower.

---

## Analogy: Down a Mountain

Imagine you want to **go down a moutain (change class in the classification)**:

With PGD/Auto-PGD (brute-force method):
- You descend always following the **steepest** slope.
- But, you can take a longer route if the seepest slope doesn't lead straight to the bottom

*Problem:*
* Sometimes the **steepest** gradient makes you take unecessary detours. You take more steps than necessary. The attack is stronger, but less optimised.

With DeepFool (Optimised Method):
- You look at the map and **find the shortest route** to the valley.
- You take only the **minimum number of steps necessary**
- You get there **faster with less effort** !


## Conclusion

DeepFool doesn't just follow the steepest descent (like PGD).

Instead, **it calculates the exact distance to the decision boundary** and makes an **optimal step size adjustment**.

Moreover, it checks the **classification at each iteration to stop his process if the classification is wrong** !

This reduces the **number of iterations** while keeping **perturbations minimal**, making the attack more efficient and harder to defend against.

---

# Code
### **AUTHOR** : Maxence QUINET (University Of Luxembourg)

## 1. Setup & Configuration

------------------

Please ensure all dependencies are installed using the `requirements.txt` file.

For additional environment setup details, refers to **"environment_configuration.txt"**.

-------------------

Below are the required **libraries and frameworks** for running Adversarial Attacks

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # (Reduce TensorFlow logs)
import tensorflow as tf
import torch
import numpy as np
import matplotlib.pyplot as plt

--------------------

**Machine Learning & Neural Network Libraries**

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import InceptionV3

------------------------
**Datasets & Image Processing**

In [ ]:
from tensorflow.keras.datasets import mnist, cifar10, cifar100
from tensorflow.keras.preprocessing.image import load_img, img_to_array

-----------------
**Adversarial Robustness Toolbox (ART)**

In [ ]:
from art.estimators.classification import TensorFlowV2Classifier, PyTorchClassifier
from art.attacks.evasion import DeepFool

------------------------------
**Vision Models**

In [ ]:
from PIL import Image

------------------------

**imagenet_stubs** 

imagenet_stubs is a small dataset available at this link : https://github.com/nottombrown/imagenet-stubs

#### Why use it ?

* Ideal for **testing adversarial attacks quickly** before applying them on larger datasets.
* Provides **two useful functions**:
  - `label_to_name(index)` --> Convert an ImageNet label (number) to its corresponding name
  - `name_to_label(name)` --> Convert an ImageNet class name back to its numerical label 

In [ ]:
import imagenet_stubs
from imagenet_stubs.imagenet_2012_labels import label_to_name, name_to_label

## Checking PyTorch & TensorFlow Environment

### **CUDA & GPU Verification**
Since we need **CUDA** for accelerated deep learning computations, we ensure that **PyTorch and TensorFlow** are properly configured with CUDA.

------------------
**PyTorch**

In [ ]:
 # Versions
print("PyTorch version:", torch.__version__)
print("TensorFlow version:", tf.__version__)

In [ ]:
# For PyTorch
print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: ', device)

------------------
**TensorFlow**

In [ ]:
# For Tensorflow
print(tf.config.list_physical_devices('GPU'))
tf.test.is_gpu_available()

In [ ]:
# Check CUDA & CUDNN Version
print("CUDA available:", tf.test.is_built_with_cuda())
print(tf.sysconfig.get_build_info()["cuda_version"])
print(tf.sysconfig.get_build_info()["cudnn_version"])

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(e)


## Dataset Selection & Configuration

In this section, you can **choose the dataset** you want to use for adversarial attacks:

- **MNIST**
- **CIFAR10**
- **CIFAR100**
- **ImageNet**

#### NOTE: On the "DeepFool Paper" they test MNIST, CIFAR10 & ImageNet datasets !

### **Select Your Dataset & Model Configuration**
Modify the variables below to **choose the dataset and model**.  

The **official DeepFool** paper test the attack on 2 models for each datasets, like LeNet, NIN, CaffeNet, GoogLeNet (Inception) !

-----------

In [ ]:
selected_dataset = "MNIST" # OPTIONS : "MNIST", "CIFAR10", "CIFAR100", and "ImageNet"

selected_cifar10_model = "standard_resnet" # OPTIONS : "standard_resnet", "resnet_10x_variant", "conv_maxout"
selected_mnist_model = "simple_cnn" # OPTIONS : "simple_cnn", "shallow_softmax", "maxout", "logistic" (For MNIST only)

selected_attack = "DeepFool" # Used for report name only.

--------------------------------------------
**Define class labels for each dataset**

In [ ]:
# Creation of the ancestors_name & ancestors_label corresponding to the selected dataset.

# Note: All labels are available on Internet. They are not created from us. They are official, often in a .json format.
if selected_dataset == "CIFAR10":
    # Correspondance between name & label for CIFAR10
    ancestors_name = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    ancestors_label = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

elif selected_dataset == "CIFAR100":
    # Correspondance between name & label for CIFAR100
    ancestors_name = ['apple', 'bridge', 'castle', 'elephant', 'house', 'orange', 'shark', 'table', 'tractor', 'whale']
    ancestors_label = ['0', '12', '17', '31', '37', '53', '73', '84', '89', '95']

elif selected_dataset == "ImageNet":
    # Correspondance between name & label for ImageNet
    ancestors_name = ['abacus', 'acorn', 'baseball', 'broom', 'brown_bear', 'canoe', 'hippopotamus', 'llama', 'maraca', 'mountain_bike']
    ancestors_label = ['398', '988', '429', '462', '294', '472', '344', '355', '641', '671']

elif selected_dataset == "MNIST":
    # Correspondance between name & label for ImageNet
    ancestors_name = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
    ancestors_label = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

else:
    print(f"Your {selected_dataset} doesn't exist. Please provide an existing dataset between these choices : CIFAR10, CIFAR100, ImageNet & MNIST.")

---------------------------------------------------------------------
**Fix seed to ensure reproducibility (comment to get random results)**

In [ ]:
np.random.seed(12345)

-----------------------

#### plot_prediction()

This function will be used to display the original / attacked images.

The function is designed to display the images correctly, depending on the dataset selected, with the following legend:

<font color='green'>Green bars</font> = correct classification <br>
<font color='red'>Red bars</font> = Attack target classification <br>
<font color='blue'>Blue bars</font> = other classifications

In [ ]:
def label_to_name_dynamic(index, dataset):
    """Retourne le nom du label en fonction du dataset sélectionné."""
    if dataset == "MNIST":
        return str(index)  # For MNIST, the label name is simply the digit
    elif dataset == "ImageNet":
        return label_to_name(index)  # Use the imagenet_stubs function for ImageNet !
    elif dataset == "CIFAR10":
        return ancestors_name[index]  # Return the name from our list
    elif dataset == "CIFAR100":
        return cifar100_labels[index] if 0 <= index < 100 else "Unknown" # Return the name from our list 
    else:
        return "Unknown"

In [ ]:
def plot_prediction(img, probs, correct_class=None, target_class=None):
    """
    Displays an image with predictions in the form of coloured bars :
    - Green --> Correct Class
    - Red --> Target Class
    - Blue --> Other Classes
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8))

    # Display the picture
    if selected_dataset=="MNIST":
        ax1.imshow(img, cmap="gray") # Force the display in gray level for MNIST !
        ax1.axis("off")
    else:
        ax1.imshow(img)
        ax1.axis("off")

    # Keep the top 10 classes with highest probabilities
    top_ten_indexes = list(probs[0].argsort()[-10:][::-1])
    top_probs = probs[0, top_ten_indexes]
    labels = [label_to_name_dynamic(i, selected_dataset) for i in top_ten_indexes]


    # Bar plot creation with color rules defined above
    barlist = ax2.bar(range(10), top_probs, color="blue")  # Blue by default

    if target_class in top_ten_indexes:
        barlist[top_ten_indexes.index(target_class)].set_color("red")  # Red if this is the target class

    if correct_class in top_ten_indexes:
        barlist[top_ten_indexes.index(correct_class)].set_color("green")  # Green if this is the correct class

    # Plot Graph
    plt.sca(ax2)
    plt.ylim([0, 1.1])
    plt.xticks(range(10), labels, rotation="vertical")
    plt.ylabel("Probability")
    plt.title("Top 10 Predictions")
    fig.subplots_adjust(bottom=0.2)

    plt.show()

## Step 1: Define Parameters

## Step 1: Define Parameters

*NOTE*: To be faster, a new parameter appear with DeepFool : `nb_grads`

As you know, **DeepFool must find the optimal direction** to get out from the current class and reach the closest boundar decision.

By default, it will compute the gradient for ALL CLASSES (ex: 1000 with ImageNet), which is obviously **very expensive in computational ressources**.

`nb_grads` will **limit this number of classes**. So if we say : `nb_grads=10`. DeepFool will consider **ONLY the 10 most probable classes instead of all classes !**

It saves a lot of time and a lot of computational ressources.

---

### **DeepFool Paper Parameters**
In the **original DeepFool paper**, they tested 4 datasets : ImageNet, MNIST, CIFAR100 & CIFAR10. 

Now, it's useless to define `epsilon_step` because it will be handle dynamically.

However, you can set up a value for `epsilon_step` . It will be considered as an `initial value`.

**You can modify these values below to test different attack configurations**

----

In [ ]:
# DeepFool Paper Research Parameters
epsilon = 8/255 # Overshoot parameter (small value to avoid going to far)
# epsilon_step = ??? # Epsilon_step is now handle dynamically since Auto-PGD
num_steps = 100 # Maximum number of iterations
# norm = np.inf # Define your norm : np.inf = L_inf / 1 = L1 / 2 = L2 --> NO NORM WITH DEEPFOOL
# restart = 1 # Number of restart that you allow --> NO RESTART WITH DEEPFOOL

nb_grads = 10 # Number of most probable class gradients to consider.

print(f"Selected Attack: {selected_attack} | Dataset: {selected_dataset} | Epsilon: {epsilon} | Epsilon Step: Dynamics | Maximum Iterations: {num_steps} | Nb_grads: {nb_grads}")

-----------------------------
**Later, we'll see what EoT is. If you don't know what is EoT, skip this sub-section**

*If you want to test EoT Transformation, find parameters below*

In [ ]:
# Parameters for EoT Transformation
angle_max = 22.5 # Rotation angle used for evaluation in degrees
eot_angle = angle_max # Maximum angle for sampling range in EoT rotation, applying range [-eot_angle, eot_angle]
eot_samples = 10 # Number of samples with random rotations in parallel per loss gradient calculation

### Dataset-Specific Parameters

In [ ]:
# ImageNet has 1000 classes, CIFAR100 100 classes, and CIFAR10 & MNIST has 10 classes.
nb_classes = 1000 if selected_dataset == "ImageNet" else 100 if selected_dataset == "CIFAR100" else 10

# ImageNet Images Dimension : (299,299,3), CIFAR10 & CIFAR100 : (32,32,3), and MNIST : (28,28,1)
input_shape = (299, 299, 3) if selected_dataset == "ImageNet" else (32, 32, 3) if "CIFAR" in selected_dataset else (28, 28, 1)

# ImageNet use often a specific preprocessing. For the others dataset, it still an adapted normalisation (0,1)
preprocessing = (0.5, 0.5) if selected_dataset == "ImageNet" else (0.0, 1.0)  # Normalisation adaptée

# Clip values 
clip_values = (0.0, 1.0)  # Same for all datasets

# Target Class Definition (You can change, here are just some examples)
if selected_dataset == "ImageNet":
    y_target = np.array([641])  # "maraca"
elif selected_dataset == "CIFAR100":
    y_target = np.array([3])  # "bear"
elif selected_dataset == "CIFAR10":
    y_target = np.array([1])  # "automobile"
else:  # MNIST
    y_target = np.array([np.random.randint(0, 10)])  # random digit between 0 and 9

## Step 2: Load Dataset Data & Labels

In this step, we **load all dataset images and their labels into memory**.

#### **How does it work?**
1. We retrive the dataset path (`datasets/selected_dataset/`).
2. We read all images from the dataset folders.
3. We **normalize** the images (scale pixel values between `[0, 1]`).
4. We store **both images and labels** for further processing.

 -------------------

In [ ]:
# List Initializations
x_all, y_all, original_images = [], [], []

In [ ]:
# Try to get our dataset path in our computer to keep all pictures and put them into our lists.
dataset_path = os.path.join("datasets", selected_dataset)
# Check
assert(dataset_path=="datasets/"+selected_dataset) # If nothing : It's ok. Otherwise, you will get an error if the dataset path doesn't exists.

In [ ]:
# Load images from the selected dataset
for class_name, class_label in zip(ancestors_name, ancestors_label):
    class_path = os.path.join(dataset_path, class_name)
    if not os.path.exists(class_path):
        continue
    
    for img_file in sorted(os.listdir(class_path)):
        img_path = os.path.join(class_path, img_file)

        if selected_dataset == "MNIST":
            im = load_img(img_path, color_mode="grayscale", target_size=(28, 28))
            im_array = img_to_array(im)
        
        elif selected_dataset == "ImageNet":
            im = load_img(img_path, target_size=(299, 299))
            im_array = img_to_array(im)

        elif selected_dataset in ["CIFAR10", "CIFAR100"]:
            im = load_img(img_path, target_size=(32, 32))
            im_array = img_to_array(im)
        
        x = (im_array / 255.0).astype(np.float32)
        
        x_all.append(x)
        y_all.append(int(class_label))
        original_images.append(im_array)

-----------------------------------------------------
#### Display Dataset (Optional)
**You can choose to display all images or only one image per class)**

#### How to enable visualization ?
- To display **ALL images** --> **Uncomment the loop bellow**.
- To display **ONLY 1 image per class** --> **Set `display_all_images = False`**.
-----------------------------------------------------

```Python
# Set to True to display all images, False to show only 1 image per class
display_all_images = False  

# Displaying of the 100 pictures (can be long, you can modify the code to display only 1 picture per class if you want)
for class_name, class_label in zip(ancestors_name, ancestors_label):
    class_path = os.path.join(dataset_path, class_name)
    if not os.path.exists(class_path):
        print(class_path)
        print("No os Path")
        continue
    
    print(f"Class : {class_name} (Label: {class_label})")
    
     # Show only 1 image per class if display_all_images = False
    images_to_show = sorted(os.listdir(class_path))[:1] if not display_all_images else sorted(os.listdir(class_path))
    # Go through the 10 pictures of each classes
    for img_file in images_to_show:
        img_path = os.path.join(class_path, img_file)

        # Load & Normalize the picture
        im = load_img(img_path, target_size=(299, 299))
        im_array = img_to_array(im)

        # Displaying all pictures
        plt.figure(figsize=(4, 4))
        plt.imshow(im_array.astype("uint8"))
        plt.axis("off")
        plt.title(f"Class: {class_name} | {img_file}", fontsize=10, fontweight="bold")
        plt.show()

        print(f"{img_file} well displayed in : {class_name}")

print(f"All of the {len(ancestors_name)} classes & their images has been displayed !")
```

### Convert to Numpy Arrays for TensorFlow
Since TensorFlow requires NumPy arrays, we convert our lists into arrays.

-----------------

In [ ]:
# Convert into a numpy array
x_all = np.array(x_all)
y_all = np.array(y_all).reshape(-1, 1)

# Check
#for img_x, img_y in zip(x_all, y_all):
#    print(f"x_all shape: {x_all.shape}")  # (N, H, W, C)
#    print(f"y_all shape: {y_all.shape}")  # (N, 1)

## Step 3 : Load Model & Loss Function

### 1. Loading Dataset for Model Training
Before creating the model, we **load and preprocess** the dataset to ensure it is correctly formatted for TensorFlow.

-------------

In [ ]:
if selected_dataset == "MNIST":
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)

elif selected_dataset == "CIFAR10":
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = to_categorical(y_train, nb_classes), to_categorical(y_test, nb_classes)
    
elif selected_dataset == "CIFAR100":
    (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode="fine")

    # We reproduce the list of all classes of CIFAR100
    cifar100_labels = [
    "apple", "aquarium_fish", "baby", "bear", "beaver", "bed", "bee", "beetle", "bicycle", "bottle",
    "bowl", "boy", "bridge", "bus", "butterfly", "camel", "can", "castle", "caterpillar", "cattle",
    "chair", "chimpanzee", "clock", "cloud", "cockroach", "couch", "crab", "crocodile", "cup", "dinosaur",
    "dolphin", "elephant", "flatfish", "forest", "fox", "girl", "hamster", "house", "kangaroo", "computer_keyboard",
    "lamp", "lawn_mower", "leopard", "lion", "lizard", "lobster", "man", "maple_tree", "motorcycle", "mountain",
    "mouse", "mushroom", "oak_tree", "orange", "orchid", "otter", "palm_tree", "pear", "pickup_truck", "pine_tree",
    "plain", "plate", "poppy", "porcupine", "possum", "rabbit", "raccoon", "ray", "road", "rocket", "rose", "sea",
    "seal", "shark", "shrew", "skunk", "skyscraper", "snail", "snake", "spider", "squirrel", "streetcar", "sunflower",
    "sweet_pepper", "table", "tank", "telephone", "television", "tiger", "tractor", "train", "trout", "tulip",
    "turtle", "wardrobe", "whale", "willow_tree", "wolf", "woman", "worm"
]

    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = to_categorical(y_train, nb_classes), to_categorical(y_test, nb_classes)

### 2. Model Selection & Architecture
On the **DeepFool** paper, they have tested the attack on LeNet, GoogLeNet(Inception), NIN, CaffeNet etc...

In [ ]:
# Definind a custom Maxout layer
class MaxoutLayer(tf.keras.layers.Layer):
    def __init__(self, num_units, **kwargs):
        super(MaxoutLayer, self).__init__(**kwargs)
        self.num_units = num_units
        self.dense = Dense(num_units * 2)  # We create twice more neurons to have the max

    def call(self, inputs):
        x = self.dense(inputs)  # Apply Linear Transformation
        x = tf.reshape(x, (-1, self.num_units, 2))  #Group neurons by pair
        return tf.reduce_max(x, axis=-1)  # Take the maximum from each pair

# Creating the Maxout model
def build_maxout_model():
    input_layer = Input(shape=(28, 28, 1))  # Input: MNIST (28, 28, 1)
    flattened = Flatten()(input_layer)

    # Add Maxout Layers
    maxout_1 = MaxoutLayer(256)(flattened)
    maxout_2 = MaxoutLayer(128)(maxout_1)

    # Output Layer
    output_layer = Dense(10, activation='softmax')(maxout_2)

    # Creating Model
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Convolutional Maxout Network
def build_conv_maxout_model(num_classes):
    input_layer = Input(shape=(32, 32, 3))  # Input CIFAR-10  et CIFAR100 (32x32, RGB) so (32, 32, 3)

    # Classic Convolutional Layers
    conv1 = Conv2D(64, (3, 3), padding="same", activation=None)(input_layer)  # No Activation (Handle by Maxout)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), padding="same", activation=None)(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Add Maxout Layers
    flattened = Flatten()(pool2)
    maxout_1 = MaxoutLayer(256)(flattened)
    maxout_2 = MaxoutLayer(128)(maxout_1)

    # Output Layer
    output_layer = Dense(num_classes, activation='softmax')(maxout_2)

    # Creating Model
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

#### The code below represent the Simple Convolutional Neural Network used in the PGD Paper for **MNIST** !

**The official code from the paper is available at this GitHub Link : https://github.com/MadryLab/mnist_challenge/blob/master/model.py**

This code is more compact because we use a higher version of tensorflow, but the model is 100% the same (you can compare if you want)

-----

In [ ]:
from tensorflow.keras import layers, models

def build_simple_CNN_model():
    model = models.Sequential([
        layers.Conv2D(32, (5, 5), activation='relu', input_shape=(28, 28, 1), padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(64, (5, 5), activation='relu', padding='same'),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        layers.Dense(10, activation='softmax')  # 10 classes for MNIST
    ])

    return model

#### The code below represent the ResNet Model & in Variant 10 times larger used in the PGD Paper for **CIFAR10** !

**The official code from the paper is available at this GitHub Link : https://github.com/MadryLab/cifar10_challenge/blob/master/model.py**

This code is more compact because we use a higher version of tensorflow, but the model is 100% the same (you can compare if you want)

----

In [ ]:
# RESNET STANDARD MODEL
def build_resnet_model():
    base_model = tf.keras.applications.ResNet50(
        include_top=False,  # Don't include the classification for ImageNet. When True --> Conserve the classification final layer pre-trained for ImageNet (1000 classes)
        input_shape=(32, 32, 3),
        weights=None  # We train the model from scratch
    )
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(10, activation='softmax')  # 10 classes for CIFAR-10
    ])
    
    return model

In [ ]:
# RESNET VARIANT 10 TIMES LARGER
def build_wide_resnet():
    base_model = tf.keras.applications.ResNet50(
        include_top=False, # Don't include the classification for ImageNet. When True --> Conserve the classification final layer pre-trained for ImageNet (1000 classes)
        input_shape=(32, 32, 3),
        weights=None
    )

    # We increase the number of filters by 10
    for layer in base_model.layers:
        if isinstance(layer, layers.Conv2D):
            layer.filters *= 10  

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(10, activation='softmax')  # 10 classes for CIFAR-10
    ])
    
    return model

In [ ]:
# ============= IMAGENET =============
if selected_dataset == "ImageNet":
    print(f"SELECTED MODEL : InceptionV3.") 
    model = InceptionV3(include_top=True, weights='imagenet', classifier_activation='softmax')
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

# ============= MNIST =============
elif selected_dataset == "MNIST":
    
    # SHALLOW MAX CLASSIFER
    if selected_mnist_model == "shallow_softmax":
        print(f"SELECTED MODEL : {selected_mnist_model}.") 
        model = Sequential([
            Flatten(input_shape=input_shape), # MNIST has a shape of (28, 28)
            Dense(128, activation='relu'),
            Dense(10, activation='softmax') # 10 classes for MNIST (0 to 9)
        ])

    # MAXOUT
    elif selected_mnist_model == "maxout":
        print(f"SELECTED MODEL : {selected_mnist_model}.")
        model = build_maxout_model()
        
    # LOGISTIC REGRESSION
    elif selected_mnist_model == "logistic":
        print(f"SELECTED MODEL : {selected_mnist_model}.")
        # Logistic Regression = one dense simple layer with softmax
        model = Sequential([
            Flatten(input_shape=(28, 28)),  # MNIST has a shape of (28, 28)
            Dense(10, activation='softmax')  # 10 classes (0-9)
        ])

    # SIMPLE CNN --> THE MODEL USED IN THE OFFICIAL PGD PAPER. OFFICIAL CODE AVAILABLE AT THIS GITHUB LINK  : https://github.com/MadryLab/mnist_challenge/blob/master/model.py
    elif selected_mnist_model == "simple_cnn":
        # Create the model
        model = build_simple_CNN_model()

        # (Optional) Print the model summary
        model.summary()
    else: 
        raise ValueError(f"Error: Model '{selected_mnist_model}' is not recognized between : shallow_softmax, maxout and logistic.")

# ============= CIFAR10/CIFAR100 =============
elif selected_dataset in ["CIFAR10", "CIFAR100"]:
    if selected_cifar10_model == "standard_resnet":
        print(f"SELECTED MODEL: Standard ResNet")
        resnet_model = build_resnet_model()
        resnet_model.summary()
        
    elif selected_cifar10_model == "resnet_x10_variant":
        print(f"SELECTED MODEL: ResNet 10x larger Variant")
        wide_resnet_model = build_wide_resnet()
        wide_resnet_model.summary()
    else:    
        print(f"SELECTED MODEL : Convolutional Maxout Network.")
        # Creating Model
        model = build_conv_maxout_model(nb_classes)

# ============= ERROR =============
else:
    raise ValueError(f"Error: Dataset '{selected_dataset} not recognized. Please ensure to use one of this dataset : ImageNet, CIFAR10, CIFAR100 or MNIST.'")


### 3. Model Compilation & Training
Once the model is selected, we **compile and train** it.

- **For ImageNet**, the model is already pretrained
- **For other datasets**, a quick training step (5-10 epochs) is performed.

------------

In [ ]:
# Compile Model
if selected_dataset != "ImageNet":
    model.compile(optimizer='adam',
              loss='categorical_crossentropy' if selected_dataset != "MNIST" else 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model 
if selected_dataset != "ImageNet":
    epochs = 5 if selected_dataset == "MNIST" else 10  # Quick training
    print(f"Training model on {selected_dataset} for {epochs} epochs...")
    model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))
    print("Model training completed.")

## Step 4 : Create the ART Classifier & Configure the Attack

Now that the model is **trained and ready**, we integrate it into **ART (Adversarial Robustness Toolbox)**.

#### What is happening here ?
1. We **create a classifier** for ART based on the trained model
2. We **define an adversarial attack** (FGSM in this case)
3. The attack can be **targeted or untargeted**, and parameters are fully configurable.

In [ ]:
classifier = TensorFlowV2Classifier(model=model,
                                    nb_classes=nb_classes,
                                    loss_object=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                                    preprocessing=preprocessing,
                                    preprocessing_defences=None,
                                    clip_values=clip_values,
                                    input_shape=input_shape)

In [ ]:
attack = DeepFool(classifier=classifier,  
                  max_iter=num_steps,  
                  epsilon=epsilon,  
                  nb_grads=nb_grads,
                  verbose=True)  # Consider only the top `nb_grads` most probable classes

## Step 5 : Predict Clean (Original) Images BEFORE the attack.

Before applying any attack, we **predict the clean images** with our trained model.

#### What happens here ?
1. We run the classifier on all images **before the attack**.
2. We display the **top-10 predictions** for each image.
3. You can choose to **display all images or only one per class**.

------------

In [ ]:
# Predict Clean Images
y_pred_clean_all = classifier.predict(np.array(x_all))

# Check prediction shape
print("Shape of Clean Predictions:", y_pred_clean_all.shape)  # Expected (N, nb_classes)

# Summarize Prediction
top1_correct = np.mean(np.argmax(y_pred_clean_all, axis=1) == y_all.flatten()) * 100
print(f"Top-1 Accuracy on Clean Images: {top1_correct:.2f}%")

---------------------------

#### Display Clean Images & Predictions
You can **choose whether to display all images or just one per class**.

**How enable visualization?**
- To display **ALL images** --> Set `display_all_images = True`
- To display **ONLY 1 image per class** --> Set `display_all_images = False`

---------------------

In [ ]:
# Set to True to display all images, False to show only 1 image per class
display_all_images = False  

# Displaying Clean Images with Predictions
for class_name, class_label in zip(ancestors_name, ancestors_label):
    print(f"\nClass : {class_name} (Label: {class_label})")

    # Get all images from this class
    class_indices = np.where(y_all == int(class_label))[0]

    if len(class_indices) == 0:
        print(f"No Images found for {class_name}, skipping...")
        continue

    # Show only 1 image per class if display_all_images = False
    images_to_show = class_indices[:1] if not display_all_images else class_indices
    
    for index in images_to_show:
        plot_prediction(
            np.squeeze(x_all[index]),  # Original clean image
            y_pred_clean_all[index].reshape(1, -1),  # Reshaped prediction
            correct_class=y_all[index],  # True class
            target_class=None  # No target class for clean images
        )
        print(f"Image {index} displayed for class: {class_name}")

print(f"\n All {len(x_all)} clean images have been processed!")

## Step 6: Generate and Evaluate Adversarial Examples

Now, we **generate adversarial examples** and evaluate the effectiveness of the attack.

 **What happens here?**
1. We **generate adversarial examples** using the selected attack.
2. We **save the adversarial images** for later analysis. (optional)
3. We **evaluate the attack's success** (accuracy, confidence scores, and performance metrics).
4. We **generate a detailed report** summarizing the attack results.

---

TensorFlow’s GradientTape is used to compute gradients efficiently. However, DeepFool requires multiple gradient calculations, which can cause high memory usage. To optimize this, we replace GradientTape with a lightweight version that improves memory management and speeds up computation.

But it could be nice to check if its works better without this new version of GradientTape !

In [ ]:
# ALLOWS YOU TO NOT HAVE THE GRADIENTTAPE WARNING, AND SAVE TIME AND CPU RESSOURCES.
# Main Improvement : Improve Memory Management.

# Modified version of GradientTape without persistent=True
class CustomGradientTape(tf.GradientTape):
    def __init__(self, persistent=False, *args, **kwargs):
        super().__init__(persistent=persistent, *args, **kwargs)

# Replace GradientTape by the modified version
tf.GradientTape = CustomGradientTape

In [ ]:
from tqdm import tqdm  # Progress bar for attack generation

target_labels = np.tile(y_target, (len(x_all), 1))  # Repeat y_target for each image (same target for all images)

if attack.targeted:
    print("SCENARIO ATTACK : TARGETED")
    # Attack all images with the target class
    x_adv_all = attack.generate(x=x_all, y=target_labels)
else:
    print("SCENARIO ATTACK : UNTARGETED")
    # No Target Class
    x_adv_all = attack.generate(x=x_all)

    
# Shape Check of the Adversarial Examples
print("Shape of Adversarial Examples:", x_adv_all.shape)  # Expected (N, H, W, C)

---------------------

**Do you want to save all adversarial images?**  
- **YES** → Uncomment the saving function below.
- **NO** → Comment the function to skip saving.


```Python
# Define the save path for adversarial images
adv_save_path = os.path.join("adversarials_img", selected_attack, selected_dataset)
os.makedirs(adv_save_path, exist_ok=True)  

# Iterate through all classes to save adversarial images
class_counters = {class_name: 1 for class_name in ancestors_name}  # Dictionary to track image indices per class

for adv_img, class_label in zip(x_adv_all, y_all.flatten()):  # Ensure y_all is 1D
    # Find the class name corresponding to the label
    if str(class_label) not in ancestors_label:
        print(f"Label {class_label} not found in ancestors_label, skipping image.")
        continue  

    class_index = ancestors_label.index(str(class_label))
    class_name = ancestors_name[class_index]

    # Determine the subfolder for the class
    class_folder = os.path.join(adv_save_path, class_name)
    os.makedirs(class_folder, exist_ok=True) 

    # Generate a unique filename with a counter (e.g., abacus1_adv.jpeg, abacus2_adv.jpeg, ..., acorn1_adv.jpeg, ...)
    img_filename = f"{class_name}{class_counters[class_name]:02d}_adv.jpeg"
    img_path = os.path.join(class_folder, img_filename)

    # Convert and save the image
    img = array_to_img(adv_img)
    img.save(img_path, "JPEG")

    print(f"Image saved : {img_path}")

    # Increment the counter for this class
    class_counters[class_name] += 1

print(f"\nAll  {len(x_adv_all)} adversarial images have been successfully saved!")
```

---

### Evaluate Adversarial Example

We now evaluate the adversarial examples by:
- Measuring the **model's accuracy** on these images.
- Computing the **confidence score** of predictions.
- Generating a **visual comparison** between clean and adversarial images.

---

**How enable visualization?**
- To display **ALL images** --> Set `display_all_images = True`
- To display **ONLY 1 image per class** --> Set `display_all_images = False`

---

In [ ]:
# Get predictions on adversarial images
y_pred_adv_all = classifier.predict(x_adv_all)

In [ ]:
# Set to True to display all images, False to show only 1 image per class
display_all_images = False  

# Display Adversarial Examples (Optional)
for class_name, class_label in zip(ancestors_name, ancestors_label):
    print(f"\nClass : {class_name} (Label: {class_label})")

    class_indices = np.where(y_all == int(class_label))[0]
    
    # Show only 1 image per class if display_all_images = False
    images_to_show = class_indices[:1] if not display_all_images else class_indices
    
    if len(class_indices) == 0:
        print(f"No images found for {class_name}skipping...")
        continue

    for index in images_to_show:
        plot_prediction(
            np.squeeze(x_adv_all[index]),
            y_pred_adv_all[index].reshape(1, -1),
            correct_class=y_all[index],
            target_class=target_labels[index]
        )
        print(f"Adversarial Image {index} displayed for class: {class_name}")

### Compute Performance Metrics

In [ ]:
# Compute confidence score
confidence_scores = np.max(y_pred_clean_all, axis=1)
average_confidence = np.mean(confidence_scores) * 100

# Compute Tok-K Accuracy
def compute_accuracy(predictions, true_labels, top_k=1):
    top_k_preds = np.argsort(predictions, axis=1)[:, -top_k:]
    match = np.any(top_k_preds == np.array(true_labels).reshape(-1, 1), axis=1)
    return np.mean(match) * 100 

In [ ]:
clean_top1 = compute_accuracy(y_pred_clean_all, y_all, top_k=1)
clean_top5 = compute_accuracy(y_pred_clean_all, y_all, top_k=5)
adv_top1 = compute_accuracy(y_pred_adv_all, y_all, top_k=1)
adv_top5 = compute_accuracy(y_pred_adv_all, y_all, top_k=5)

In [ ]:
# Display Performance Results
attack_name = "deepfool" if isinstance(attack, DeepFool) else "fast"

In [ ]:
print("\n=== Performance Summary ===")
print(f"Selected Attack: {selected_attack} | Dataset: {selected_dataset} | Epsilon: {epsilon} | Epsilon Step: Dynamics | Number of Iterations: {num_steps}")
print(f"Clean Images : Top-1 : {clean_top1:.2f}% | Top-5 : {clean_top5:.1f}%")
print(f"Adv. Images  : Top-1 : {adv_top1:.2f}% | Top-5 : {adv_top5:.1f}%")
print("----------------------------------------------------------")
print(f"Confidence Score: {average_confidence:.2f}%")

### Generate a Report

In [ ]:
# Round epsilon for eadability
eps_rounded = round(epsilon, 3)

# Define report save path
if selected_dataset == "MNIST":
    report_filename = f"{selected_attack}_with_{selected_dataset}_with_{selected_mnist_model}_report_eps={eps_rounded}_iteration={num_steps}.txt"
elif selected_dataset in ["CIFAR10", "CIFAR100"]:
    report_filename = f"{selected_attack}_with_{selected_dataset}_with_{selected_cifar10_model}_report_eps={eps_rounded}_iteration={num_steps}.txt"
elif selected_dataset == "ImageNet":
    report_filename = f"{selected_attack}_with_{selected_dataset}_with_InceptionV3_report_eps={eps_rounded}.txt"
else:
    print(f"This {selected_dataset} is not recognized. Be careful to provide an existing dataset between MNIST, CIFAR")
    
report_path = os.path.join("adversarials_img", selected_attack, selected_dataset, report_filename)
os.makedirs(os.path.dirname(report_path), exist_ok=True)


with open(report_path, "w", encoding="utf-8") as f:
    # Report Title
    f.write(f"====== {selected_attack} Adversarial Attack Report (ε = {eps_rounded}) ======\n\n")

    # Information generation for each image
    for i in range(len(y_pred_adv_all)):  
        # Find the class index in ancestors_label
        class_label = str(y_all[i][0])  # Convert to string to match ancestors_label
        if class_label in ancestors_label:
            class_index = ancestors_label.index(class_label)  # Get index in ancestors_name
            class_name = ancestors_name[class_index]  # Retrieve class name
        else:
            class_name = "Unknown"  # If not found, prevent error

        # Original Image file name (ensuring correct numbering)
        original_image_name = f"{class_name}{(i % 10) + 1:02d}.jpeg"

        # Predict Class for the original image (top-1)
        clean_pred_index = np.argmax(y_pred_clean_all[i])

        # Predict Class for the Adversarial image (top-1)
        adv_pred_index = np.argmax(y_pred_adv_all[i])

        # Prediction
        clean_pred_label = label_to_name_dynamic(clean_pred_index, selected_dataset)
        adv_pred_label = label_to_name_dynamic(adv_pred_index, selected_dataset)


        # Targeted or Untargeted Scenario Attack
        attack_type = "Targeted" if attack.targeted else "Untargeted"

        # If Targeted : Target Class
        target_label = label_to_name(y_target[0]) if attack.targeted else "N/A"

        # Write results in the report:
        f.write(f"------ CLASS : {class_name.upper()} ------\n")
        f.write(f"Original image name : {original_image_name}\n")
        f.write(f"Original Prediction : {clean_pred_label}\n")
        f.write(f"Targeted / Untargeted : {attack_type}\n")
        if attack.targeted:
            f.write(f"Target Class : {target_label}\n")
        f.write(f"Adversarial Prediction : {adv_pred_label}\n")
        f.write("------------------------------------------------\n\n")

    # Performance Summary at the end of the file
    f.write("============ PERFORMANCE RESUME ============\n")
    f.write(f"Selected Attack: {selected_attack} | Dataset: {selected_dataset} | Epsilon: {eps_rounded} | Epsilon Step: Dynamics | Number of Iterations: {num_steps}\n")
    f.write(f"Clean Images : Top-1 : {clean_top1:.1f}% | Top-5 : {clean_top5:.1f}%\n")
    f.write(f"Adv. Images  : Top-1 : {adv_top1:.1f}% | Top-5 : {adv_top5:.1f}%\n")
    f.write("----------------------------------------------------------")
    f.write(f"Confidence Score: {average_confidence:.2f}%")

    attack_eff_top1 = 100 - adv_top1
    attack_eff_top5 = 100 - adv_top5

    f.write("\n")
    f.write(f"{selected_attack} Efficiency : Top-1 : {attack_eff_top1:.1f}% | Top-5 : {attack_eff_top5:.1f}%\n")

# Saving Confirmation
print(f"Report saved : {report_path}")

# Going further (optional) : Expectation Over Transformation (EoT) 
Adversarial attacks like **FGSM** are often **sensitive to image transformations** such as **rotation, scaling, or noise**.

**Why does this happen?**  
- A small rotation (e.g., **5°**) can **invalidate** an adversarial example.
- This **breaks the perturbation pattern** that misleads the classifier.
  
**How does EoT (Expectation Over Transformation) help?**  
- Instead of using **a single perturbed image**, EoT **randomly transforms** the image (rotation, blur, etc.).
- The attack is then **optimized over multiple transformations**, making it **more robust**.

---

In [ ]:
import scipy.ndimage

# Define rotation angles to test
rotation_angles = [-22.5, -10.0, -5.0, 0.0, 5.0, 10.0, 22.5]  

# Apply rotation to all adversarial examples
x_adv_rotated_all = {
    angle: np.array([
        scipy.ndimage.rotate(img, angle=angle, reshape=False, axes=(0, 1), order=1, mode='constant')
        for img in x_adv_all
    ]) for angle in rotation_angles
}

# Get predictions after rotation
y_pred_adv_rotated_all = {
    angle: classifier.predict(x_adv_rotated_all[angle])
    for angle in rotation_angles
}

print(f"Adversarial images rotated and evaluated for {len(rotation_angles)} angles.")


### Display Rotated Adversarial Examples
You can **choose whether to display all images or just a few.**

In [ ]:
display_all_images = False  # Set to True to display all, False to show a few per angle

for angle in rotation_angles:
    print(f"\nRotation Angle: {angle}°")

    for i in range(len(x_adv_rotated_all[angle])):
        if not display_all_images and i > 1:
            break

        plot_prediction(
            np.squeeze(x_adv_rotated_all[angle][i]),  
            y_pred_adv_rotated_all[angle][i].reshape(1, -1),  
            correct_class=y_all[i],  
            target_class=y_target  
        )

### Evaluate Performance After Rotation

In [ ]:
# Compute Accuracy After Rotation
for angle in rotation_angles:
    adv_top1_rotated = compute_accuracy(y_pred_adv_rotated_all[angle], y_all, top_k=1)
    adv_top5_rotated = compute_accuracy(y_pred_adv_rotated_all[angle], y_all, top_k=5)
    
    print(f"Rotation {angle}° → Top-1: {adv_top1_rotated:.1f}% | Top-5: {adv_top5_rotated:.1f}%")

## Step 7: Apply Expectation Over Transformation (EoT)

### **What is EoT and Why is it Useful?**
FGSM and adversarial attacks often **fail** when images undergo transformations like **rotations**.

**EoT (Expectation Over Transformation) mitigates this issue by:**
- Generating multiple **randomly transformed** versions of the adversarial image.
- Applying these transformations **during model evaluation** (predictions & gradients).
- Making the adversarial attack **robust to transformations** like **rotations, noise, and blur**.

---

### **Enable EoT in ART**
We use ART’s **`EoTImageRotationTensorFlow`** to introduce **random rotations** during classification.

---


In [ ]:
# Create ART Classifier with EoT
eot_rotation = EoTImageRotationTensorFlow(nb_samples=eot_samples,  
                                          clip_values=clip_values,  
                                          angles=eot_angle)  # Random rotation range

classifier_eot = TensorFlowV2Classifier(model=model,
                                        nb_classes=nb_classes,
                                        loss_object=tf.keras.losses.CategoricalCrossentropy(),
                                        preprocessing=preprocessing,
                                        preprocessing_defences=[eot_rotation],  # EoT applied
                                        clip_values=clip_values,
                                        input_shape=input_shape)

print(f"EoT Classifier created with {eot_samples} transformation samples per evaluation.")


### Generate Adversarial Examples with EoT
We generate **adversarial examples** that remain effective even **after transformations**.

-----------------

In [ ]:
from tqdm import tqdm

# Prepare target labels for targeted attacks
y_target_one_hot = np.zeros((1, nb_classes), dtype=np.float32)
y_target_one_hot[0, name_to_label("guacamole")] = 1.0  
y_target_all = np.tile(y_target_one_hot, (len(x_all), 1))  

x_adv_eot_all = []

for i in tqdm(range(len(x_all)), desc="Generating EoT Examples"):
    x_i = np.expand_dims(x_all[i], axis=0)  
    y_i = np.expand_dims(y_target_all[i], axis=0)  

    if attack.targeted:
        x_adv_i = attack.generate(x=x_i, y=y_i)
    else:
        x_adv_i = attack.generate(x=x_i)

    x_adv_eot_all.append(np.squeeze(x_adv_i))  

x_adv_eot_all = np.array(x_adv_eot_all)

print(f"Shape of EoT Adversarial Examples: {x_adv_eot_all.shape}")

### Apply Rotation to Adversarial Examples
We now test the **robustness** of these adversarial examples by **rotating them** at different angles.

--------------------

In [ ]:
# Define rotation angles
rotation_angles = [-22.5, -10.0, -5.0, 0.0, 5.0, 10.0, 22.5]  

# Rotate and Evaluate Adversarial Examples
x_adv_rotated_all = {
    angle: np.array([
        scipy.ndimage.rotate(img, angle=angle, reshape=False, axes=(1, 2), order=1, mode='constant')
        for img in x_adv_eot_all
    ]) for angle in rotation_angles
}

y_pred_adv_rotated_all = {
    angle: classifier.predict(x_adv_rotated_all[angle])
    for angle in rotation_angles
}

print(f"Adversarial images rotated and evaluated for {len(rotation_angles)} angles.")

### Display Rotated Adversarial Examples
You can **choose whether to display all images or just a few per rotation angle**.

----------

In [ ]:
display_all_images = False  

for angle in rotation_angles:
    print(f"\nRotation Angle: {angle}°")

    for i in range(len(x_adv_rotated_all[angle])):
        if not display_all_images and i > 1:  
            break

        plot_prediction(
            np.squeeze(x_adv_rotated_all[angle][i]),  
            y_pred_adv_rotated_all[angle][i].reshape(1, -1),  
            correct_class=y_all[i],  
            target_class=y_target  
        )

### Evaluate Performance After Rotation

In [ ]:
# Compute Accuracy After Rotation
for angle in rotation_angles:
    adv_top1_rotated = compute_accuracy(y_pred_adv_rotated_all[angle], y_all, top_k=1)
    adv_top5_rotated = compute_accuracy(y_pred_adv_rotated_all[angle], y_all, top_k=5)
    
    print(f"Rotation {angle}° → Top-1: {adv_top1_rotated:.1f}% | Top-5: {adv_top5_rotated:.1f}%")

### Generate a Report on EoT Performance

In [ ]:
# Define report save path
report_filename = f"EoT_{selected_attack}_{selected_dataset}_eps={round(epsilon, 2)}.txt"
report_path = os.path.join("adversarials_img", selected_attack, selected_dataset, report_filename)
os.makedirs(os.path.dirname(report_path), exist_ok=True)

# Generate Report
print("\nGenerating EoT attack report...")

with open(report_path, "w", encoding="utf-8") as f:
    f.write(f"====== EoT Adversarial Attack Report (ε = {round(epsilon, 2)}) ======\n\n")
    
    for angle in rotation_angles:
        adv_top1_rotated = compute_accuracy(y_pred_adv_rotated_all[angle], y_all, top_k=1)
        adv_top5_rotated = compute_accuracy(y_pred_adv_rotated_all[angle], y_all, top_k=5)
        
        f.write(f"\n=== Rotation {angle}° ===\n")
        f.write(f"Top-1 Accuracy: {adv_top1_rotated:.1f}%\n")
        f.write(f"Top-5 Accuracy: {adv_top5_rotated:.1f}%\n")

print(f"EoT Report saved: {report_path}")